<a href="https://colab.research.google.com/github/perch97/Machine_Learning_HSE/blob/main/LogReg_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [21]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [22]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [23]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [24]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [25]:
# ваш код здесь
data.describe(include = 'object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [26]:
# ваш код здесь
from sklearn.preprocessing import OneHotEncoder

In [27]:
ohe = OneHotEncoder()

In [28]:
feature_array = ohe.fit_transform(data[['Marital Status','Gender','Home Owner','Purchased Bike']]).toarray()

In [29]:
feature_labels = ohe.categories_
print(feature_labels)

[array(['Married', 'Single'], dtype=object), array(['Female', 'Male'], dtype=object), array(['No', 'Yes'], dtype=object), array(['No', 'Yes'], dtype=object)]


In [30]:
feature_labels = np.array(feature_labels).ravel()

In [31]:
feature_labels[-1] = 'Bike_Purchased_Yes'

In [32]:
feature_labels[-3] = 'Home_Owner_Yes'

In [33]:
df_features = pd.DataFrame(data = feature_array,columns = feature_labels)

In [34]:
data = pd.concat([data,df_features], axis = 1)

Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [35]:
# ваш код здесь
data.drop(columns = ['Marital Status','Gender','Education','Occupation','Home Owner','Commute Distance','Region'],inplace = True)


In [36]:
data.drop(columns = ['ID'],inplace = True)

In [37]:
data

,Income,Children,Cars,Age,Purchased Bike,Married,Single,Female,Male,No,Home_Owner_Yes,No,Bike_Purchased_Yes
0,40000,1,0,42,No,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,30000,3,1,43,No,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,80000,5,2,60,No,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,70000,0,1,41,Yes,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,30000,0,0,36,Yes,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,60000,2,2,54,Yes,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
996,70000,4,0,35,Yes,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
997,60000,2,0,38,Yes,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
998,100000,3,3,38,No,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0


In [38]:
data.drop(columns = ['Married','Female','No','No'],inplace = True)

In [39]:
data.drop(columns = ['Purchased Bike'],inplace = True)

Удалите столбец `ID`, так как он по сути нечисловой.

In [40]:
# ваш код здесь


Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [41]:
# ваш код здес
X = data.iloc[:,:-1]
y = data['Bike_Purchased_Yes']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [42]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.3, random_state = 42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [43]:
# ваш код здесь
Xtrain

,Income,Children,Cars,Age,Single,Male,Home_Owner_Yes
541,50000,1,0,34,0.0,1.0,1.0
440,90000,0,3,34,1.0,1.0,0.0
482,40000,0,0,36,1.0,1.0,0.0
422,110000,0,3,32,1.0,1.0,0.0
778,90000,5,3,41,0.0,1.0,1.0
...,...,...,...,...,...,...,...
106,70000,2,2,52,0.0,1.0,1.0
270,10000,2,0,51,1.0,0.0,1.0
860,30000,0,1,32,1.0,1.0,1.0
435,10000,2,1,68,1.0,0.0,0.0


Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [44]:
from sklearn.preprocessing import MinMaxScaler

# ваш код здесь
scaler = MinMaxScaler()

scaler.fit(Xtrain)

Xtrain_scaled = scaler.transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

In [45]:
Xtrain = pd.DataFrame(Xtrain_scaled, columns = ['Income','Children','Cars','Age','Single','Male','Home_Owner_Yes'])

In [46]:
Xtest = pd.DataFrame(Xtest_scaled, columns = ['Income','Children','Cars','Age','Single','Male','Home_Owner_Yes'])

Теперь обучите логистическую регрессию на тренировочных данных

In [47]:
from sklearn.linear_model import LogisticRegression

# ваш код здесь
lr = LogisticRegression()
lr.fit(Xtrain,ytrain)
prediction = lr.predict(Xtest)
probs_test = lr.predict_proba(Xtest)


Сделайте предсказания на тренировочных и на тестовых данных.

In [48]:
# ваш код здесь

Оцените значение accuracy на трейне и на тесте.

In [49]:
from sklearn.metrics import accuracy_score

accuracy_score(ytest,prediction)

0.58

Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [50]:
from sklearn.preprocessing import PolynomialFeatures

# ваш код здесь

poly = PolynomialFeatures(degree=2, interaction_only=False)
poly.fit(Xtrain)


PolynomialFeatures()

In [51]:
Xtrain_featured = pd.DataFrame(poly.transform(Xtrain))

In [52]:
Xtest_featured = pd.DataFrame(poly.transform(Xtest))

)**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [53]:
# ваш код здесь
lr_updated = LogisticRegression()
lr_updated.fit(Xtrain_featured,ytrain)
prediction_updated = lr_updated.predict(Xtest_featured)
probs_test_updated = lr_updated.predict_proba(Xtest_featured)


In [54]:
accuracy_score(ytest,prediction_updated)

0.63

**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [55]:
# ваш код здесь
from sklearn.metrics import recall_score
recall_score(ytest, prediction_updated)


0.5460526315789473

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [66]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 7), (210, 7))

In [67]:
logreg = LogisticRegression()
logreg.fit(XtrainS,ytrainS)
pred = logreg.predict(Xval)
probs = logreg.predict_proba(Xval)


In [76]:
yval

375    0.0
556    0.0
610    0.0
634    0.0
157    0.0
      ... 
530    1.0
387    1.0
62     1.0
605    1.0
4      1.0
Name: Bike_Purchased_Yes, Length: 210, dtype: float64

In [72]:
(probs >= 0.2).astype('int')

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [90]:
# Extracting predicted probability of class 1
y_predict_prob_class_1 =probs[:,1]
# Define threshold 0.4
y_predict_class = [1 if prob > 0.4 else 0 for prob in y_predict_prob_class_1]

print(y_predict_prob_class_1)

[0.33921231 0.69254884 0.53899565 0.28974361 0.52297455 0.32491341
 0.23592036 0.48973386 0.51707366 0.33214978 0.4271352  0.7211509
 0.60937433 0.30858783 0.75644026 0.47823771 0.34582983 0.45013209
 0.52003424 0.39846807 0.30887396 0.36500319 0.40692    0.26348937
 0.38457854 0.58050442 0.28303833 0.57054904 0.23706893 0.58019195
 0.34111041 0.32082284 0.27916309 0.40948725 0.31704527 0.37630164
 0.71259575 0.5024902  0.40656765 0.48943791 0.58278374 0.66023446
 0.66110707 0.57470649 0.73213809 0.22034977 0.36246226 0.47823771
 0.60370862 0.37772595 0.55464528 0.57857606 0.49169423 0.38329885
 0.55963759 0.2494211  0.22924662 0.35097156 0.3679608  0.5856303
 0.53506051 0.44878193 0.18991018 0.42457515 0.34292033 0.36304796
 0.64694046 0.31701377 0.42514857 0.64567318 0.42331974 0.667299
 0.58178436 0.37141643 0.51769705 0.41072412 0.26396171 0.56697041
 0.57733499 0.47408303 0.42608891 0.58377965 0.42228556 0.61626424
 0.88255609 0.46295337 0.57246283 0.60683746 0.70825911 0.54167277

In [120]:
from sklearn.metrics import recall_score
y_predict_prob_class_1 =probs[:,1]
y_predict_prob_class_0 = probs[:,0]
RecMax = -1
BestThr = -1
BestAcc = -1

acc_rec = []

for thr in np.arange(0, 1, 0.01):
  y_predict_class = [1 if prob > thr else 0 for prob in y_predict_prob_class_1]
  acc_score =  accuracy_score(yval,y_predict_class)
  rec_score = recall_score(yval,y_predict_class)
  acc_rec.append([thr,acc_score,rec_score])

# print(len(acc_rec))



# print (BestThr, RecMax, BestAcc)

In [121]:
for i in acc_rec:
  if i[1]>=0.6:
    print(i)

[0.34, 0.6, 0.8529411764705882]
[0.35000000000000003, 0.6095238095238096, 0.8333333333333334]
[0.36, 0.6142857142857143, 0.8333333333333334]
[0.37, 0.6, 0.7941176470588235]
[0.38, 0.6095238095238096, 0.7745098039215687]
[0.39, 0.6142857142857143, 0.7450980392156863]
[0.4, 0.6190476190476191, 0.7450980392156863]
[0.41000000000000003, 0.6238095238095238, 0.7352941176470589]
[0.42, 0.6047619047619047, 0.6862745098039216]
[0.44, 0.6047619047619047, 0.6176470588235294]
[0.47000000000000003, 0.6095238095238096, 0.5784313725490197]
[0.48, 0.6, 0.5294117647058824]
[0.49, 0.6190476190476191, 0.5196078431372549]
[0.5, 0.6142857142857143, 0.5]
[0.51, 0.6095238095238096, 0.49019607843137253]
[0.52, 0.6095238095238096, 0.4803921568627451]
[0.53, 0.6095238095238096, 0.45098039215686275]
[0.54, 0.6, 0.4117647058823529]
[0.55, 0.6, 0.4019607843137255]
[0.56, 0.6047619047619047, 0.38235294117647056]
[0.5700000000000001, 0.6, 0.3627450980392157]


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [100]:
# ваш код здесь
logreg_original = LogisticRegression()
logreg_original.fit(Xtrain,ytrain)
predict_original = logreg_original.predict(Xtest)
recall_score(ytest,predict_original)

0.45394736842105265

[Текст ссылки](https://)**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.

In [113]:
probabilities_test = logreg_original.predict_proba(Xtest)

In [122]:
round(recall_score(ytest,np.where(probabilities_test[:,1]>0.35,1,0)),1)

0.8